<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados. Modificado el 2018-1, 2018-2 por Equipo docente IIC2233</font>
</p>

## Diccionarios

Imaginemos que estamos escribiendo un programa para manejar dinero, y necesitamos saber qué moneda usa el usuario dado el país en que se encuentra. Una posible solución es escribir una función que reciba el nombre del país y que en una trama de `if - elif - else` retorne la moneda correspondiente:

In [2]:
def get_moneda(pais):
    if pais == "Chile":
        return "Peso"
    elif pais == "Perú":
        return "Soles"
    elif pais == "España" or pais == "Holanda":
        return "Euro"
    elif pais == "Brasil":
        return "Real"

No obstante, esta solución es larga, tediosa de escribir, y muy difícil de mantener. Tampoco nos permite extender o actualizar la información en tiempo de ejecución, con datos que podemos leer desde un archivo o desde un servicio web. Si nos fijamos, lo único que estamos haciendo es asociar un valor que conocemos de antemano con otro valor. Es esto lo que hacen precisamente los **diccionarios**.

Un **diccionario** es una estructura de datos **ordenada** (nota importante: es ordenada sólo [a partir de Python 3.6](https://docs.python.org/3/whatsnew/3.6.html#whatsnew36-compactdict)) y **mutable** que permite asociar pares de elementos mediante la relación **llave-valor**. Al diccionario se le consulta por una **llave** y retorna su **valor** asociado. La **llave** y el **valor** (_key-value_) pueden tener distinto tipo en un diccionario. Es importante mencionar que esta estructura de tipo _mapping_ (porque _mapean_ de un valor a otro) existe en muchos otros lenguajes de programación con otros nombres distintos a _diccionario_, y podrían no preservar el orden (como lo hacía Python antes de su versión 3.6).

En el ejemplo de las monedas, los países serían las llaves por las que consultamos, y los valores serían las distintas monedas.

![](img/hash-table.png)

Dado una llave, **buscar su valor** en un diccionario es una operación **muy eficiente** que toma tiempo constante, es decir, no depende de la cantidad de elementos que tenga la estructura. No obstante, esta estructura no está diseñada para buscar (en forma eficiente) una llave teniendo el valor.

Los diccionarios están implementados con una estructura llamada _tabla de hash_. En esta estructura, existe una función matemática que se le aplica a la llave para saber en qué lugar guardar un determinado valor. Esa función se llama _función de hash_.

### Llaves permitidas

No todo elemento puede ser usado como llave en un diccionario. El primer requisito es que las llaves **deben ser únicas** (no pueden repetirse), de lo contrario podría pasar que dos valores queden asociados a la misma llave. El segundo requisito es que la llave debe ser [_hasheable_](https://docs.python.org/3/glossary.html#term-hashable).

Un objeto es _hasheable_ si:
1. Implementa la función [`__hash__`](https://docs.python.org/3/reference/datamodel.html#object.__hash__). Esta función retorna un entero y sirve de entrada para la función de _hash_ de la _tabla de hash_.
2. Implementa la función [`__eq__`](https://docs.python.org/3/reference/datamodel.html#object.__eq__). Esta función compara dos objetos y retorna `True` si éstos deben ser considerados iguales.
2. El valor que retorna `__hash__` **no cambia** durante el ciclo de vida del objeto.
3. Si dos objetos son iguales según la función `__eq__`, entonces el valor que retorna `__hash__` debe ser el mismo. No es necesario que esto se cumpla al revés. 

En Python en particular, todos los _built-ins_ que son **inmutables** son _hasheables_. Esto significa que tipos como `int`, `str` o `tuple` se pueden usar como llave en un diccionario. Por el contrario, todos los _built-ins_ mutables no son _hasheables_, por lo que tipos como `list` no pueden ser ocupados como llave.

En el ejemplo de abajo, inicializamos dos _strings_ iguales. Primero, vemos que su valor de _hash_ es igual. También vemos que son iguales. Recordemos que el hecho de que dos objetos sean iguales **no significa** que sean el mismo, esto lo podemos ver usando la sentencia `is` que retorna `True` sólo si dos variables apuntan al mismo objeto.

In [3]:
hello_1 = "Hello world"
hello_2 = "Hello world"

print(f"Hash de hello_1: {hash(hello_1)}")
print(f"Hash de hello_2: {hash(hello_2)}")
print(f"¿Son iguales? {hello_1 == hello_2}")
print(f"¿Son el mismo objeto? {hello_1 is hello_2}")

Hash de hello_1: -1768473161
Hash de hello_2: -1768473161
¿Son iguales? True
¿Son el mismo objeto? False


Notar que una tupla es _hasheable_ sólo si todos los valores que contiene son _hasheables_. Por ejemplo, una tupla que contiene una lista en su interior no es _hasheable_.

In [4]:
tupla = (0, 1, [2, 3])
hash(tupla)

TypeError: unhashable type: 'list'

Las instancias de clases creadas por el usuario son _hasheables_ por defecto, puesto que la función `__hash__` retorna un valor único y fijo para cada instancia. Además, la función `__eq__` devuelve `True` sólo si corresponde exactamente al mismo objeto, por lo que no necesitamos que el valor de `__hash__` sea igual al de ningún otro objeto.

Podemos ver que los objetos de clases personalizadas son _hasheables_ incluso si almacenan valores no _hasheables_ en alguno de sus atributos:

In [7]:
class Dummy:
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
dummy_0 = Dummy(4, 5)
dummy_1 = Dummy(0, [1, 2])
dummy_2 = Dummy("Hello", ["World", "Hold on"])

print(f"Hash de dummy_0: {hash(dummy_0)}")
print(f"Hash de dummy_1: {hash(dummy_1)}")
print(f"Hash de dummy_2: {hash(dummy_2)}")

Hash de dummy_0: 1031709
Hash de dummy_1: 1031701
Hash de dummy_2: 6476031


Esto ocurre porque, en instancias, el valor del _hash_ **no depende** de los valores de los atributos de la instancia.

In [8]:
dummy_1.x = 33
print(f"Hash de dummy_1: {hash(dummy_1)}")

Hash de dummy_1: 1031701


Por último, los valores del _hash_ para cada objeto son únicos (aunque los valores de los atributos sean iguales).

In [9]:
dummy_3 = Dummy("Hello", ["World", "Hold on"])
print(f"Hash de dummy_3: {hash(dummy_3)}")

Hash de dummy_3: 6475957


### Valores permitidos

No hay restricciones respecto a los valores. Los valores pueden ser mutables o inmutables, _hasheables_ o no, e incluso pueden incluir otros diccionarios.

### Diccionarios en Python

En Python, los diccionarios están implementados por la case `dict`. La notación para describir un diccionario es con `{}`, y cada par llave-valor se asocia con `:` como muestra el ejemplo:

In [7]:
monedas = {"Chile": "Peso", "Perú": "Soles", "España": "Euro", 
           "Holanda": "Euro", "Brasil": "Real"}

Para acceder al valor asociado a una llave, utilizamos la instrucción `diccionario[nombre_llave]`:

In [8]:
monedas["Chile"]

'Peso'

Si se intenta consultar por una llave que no existe, obtenemos un error:

In [9]:
monedas["Argentina"]

KeyError: 'Argentina'

Otra manera de acceder al valor asociado a una llave consiste en utilizar el método [`get`](https://docs.python.org/3/library/stdtypes.html#dict.get) que posee la clase diccionario. Este método requiere dos parámetros: la llave buscada y un valor en caso que la llave no exista.

In [10]:
print(monedas.get('Chile', 'No tiene moneda'))
print(monedas.get('Perú', 0))
print(monedas.get('Argentina', 'No tiene moneda'))
print(monedas.get('Colombia', False))

Peso
Soles
No tiene moneda
False


Como los diccionarios son **mutables**, si se asigna un valor a una llave existen dos comportamientos posibles. Si la llave no existe, ésta se crea y se le asigna un valor.

In [11]:
monedas["Vaticano"] = "Lira"
monedas

{'Chile': 'Peso',
 'Perú': 'Soles',
 'España': 'Euro',
 'Holanda': 'Euro',
 'Brasil': 'Real',
 'Vaticano': 'Lira'}

Si la llave ya existe, se actualiza con el nuevo valor:

In [12]:
monedas["Vaticano"] = "Euro"
monedas

{'Chile': 'Peso',
 'Perú': 'Soles',
 'España': 'Euro',
 'Holanda': 'Euro',
 'Brasil': 'Real',
 'Vaticano': 'Euro'}

A diferencia de otros lenguajes como C# o Java, en Python no es necesario que las llaves sean todas del mismo tipo. Tampoco es necesario que los valores sean todos del mismo tipo.

In [13]:
monedas[3.14] = ("Peso", "Dolar")
monedas

{'Chile': 'Peso',
 'Perú': 'Soles',
 'España': 'Euro',
 'Holanda': 'Euro',
 'Brasil': 'Real',
 'Vaticano': 'Euro',
 3.14: ('Peso', 'Dolar')}

Se puede eliminar items desde el diccionario utilizando la sentencia `del` como `del diccionario[<llave>]`.

In [14]:
del monedas[3.14]
monedas

{'Chile': 'Peso',
 'Perú': 'Soles',
 'España': 'Euro',
 'Holanda': 'Euro',
 'Brasil': 'Real',
 'Vaticano': 'Euro'}

Se puede comprobar la existencia de una llave en el diccionario utilizando la sentencia `in`. El comportamiento por defecto al utilizar sentencias sobre el diccionario es operar sobre los valores de las llaves. En el caso de `in`, devuelve `True` si la llave requerida existe dentro de las llaves en el diccionario.

In [15]:
print('Chile' in monedas)
print('Argentina' in monedas)
print('Peso' in monedas)

True
False
False


#### Métodos útiles

Tres métodos útiles que existen en los diccionarios son: 

1. `keys()`: permite obtener una lista con las llaves del diccionario.
2. `values()`: permite obtener una lista con los valores del diccionario.
3. `items()`: permite obtener una lista con los **pares** que tiene el diccionario. Cada par es una tupla de la forma `(llave, valor)`.

In [16]:
print(monedas.keys()) # una lista con todas las llaves
print(monedas.values()) # una lista con todos los valores
print(monedas.items()) # una lista con tuplas de pares llave-valor

dict_keys(['Chile', 'Perú', 'España', 'Holanda', 'Brasil', 'Vaticano'])
dict_values(['Peso', 'Soles', 'Euro', 'Euro', 'Real', 'Euro'])
dict_items([('Chile', 'Peso'), ('Perú', 'Soles'), ('España', 'Euro'), ('Holanda', 'Euro'), ('Brasil', 'Real'), ('Vaticano', 'Euro')])


Estos métodos son prácticos y útiles durante la iteración sobre diccionarios.

In [17]:
print('Las llaves en el diccionario son las siguientes:')

for m in monedas.keys():
    print(f'{m}')

print()

for m in monedas: # por defecto, se recorren las llaves
    print(f'{m}')

Las llaves en el diccionario son las siguientes:
Chile
Perú
España
Holanda
Brasil
Vaticano

Chile
Perú
España
Holanda
Brasil
Vaticano


In [18]:
print('Los valores en el diccionario:')
for v in monedas.values():
    print(f'{v}')

Los valores en el diccionario:
Peso
Soles
Euro
Euro
Real
Euro


In [19]:
print('Los pares en el diccionario:')
for k, v in monedas.items():
    print(f'La moneda de {k} es {v}')

Los pares en el diccionario:
La moneda de Chile es Peso
La moneda de Perú es Soles
La moneda de España es Euro
La moneda de Holanda es Euro
La moneda de Brasil es Real
La moneda de Vaticano es Euro


### Aplicaciones

Una aplicación de diccionarios es para realizar conteos de frecuencia. En el siguiente ejemplo, el código cuenta cuántas veces aparece cada vocal en un _string_.

In [20]:
msg = 'supercalifragilisticoespialidoso'
# Crea un diccionario vacío para contabilizar las letras
vocales = dict() 

for letra in msg:
    # Revisa si la letra es una vocal
    if letra not in 'aeiou': 
        continue
    
    # Revisa si letra existe en el diccionario, si no la crea en 0
    if letra not in vocales: 
        vocales[letra] = 0

    vocales[letra] += 1 # si ya existe, agrega una cuenta mas

print(vocales)

{'u': 1, 'e': 2, 'a': 3, 'i': 6, 'o': 3}


## `defaultdicts`

Los `defaultdicts` son diccionarios que nos permiten asignar un valor por defecto a cada *key* con la que se llama el diccionario. Esto nos ahorra el problema de tener que escribir código preocupándonos de los casos en que el valor que se intenta obtener el diccionario no existe. 

Los `defaultdicts` reciben una función (o un _callable_) que debe devolver el valor que se asignará por defecto. Esta función _no debe recibir parámetros_, puede realizar cualquier acción, y puede devolver cualquier objeto, el cual será asignado como valor para el respectivo *key* en el diccionario.
 
Por ejemplo, podemos rehacer el ejemplo de contar las vocales de un _string_.

In [21]:
from collections import defaultdict

msg = 'supercalifragilisticoespialidoso'

# Crea un defaultdict vacío.
vocales = defaultdict(int)  
# Pasamos int como callable. El callable se va a llamar sin parámetros
# cada vez que se consulte por una key que no existe.
# En este caso, int() devolverá el valor por defecto de este tipo (0)

for letra in msg:
    if letra not in 'aeiou': # Revisa si la letra es una vocal
        continue

    vocales[letra] += 1 # si ya existe, agrega una cuenta mas

print(vocales)

defaultdict(<class 'int'>, {'u': 1, 'e': 2, 'a': 3, 'i': 6, 'o': 3})


También podemos entregar una función que nosotros definamos. En este ejemplo, cada vez que una *key* no exista le asociará un número _random_.

In [22]:
from random import random

def funcion_default():
    return random()

diccionario = defaultdict(funcion_default)

print(diccionario)
diccionario['A']
print(diccionario)
diccionario['B']
print(diccionario)

defaultdict(<function funcion_default at 0x7f65b8257a60>, {})
defaultdict(<function funcion_default at 0x7f65b8257a60>, {'A': 0.8460487629107208})
defaultdict(<function funcion_default at 0x7f65b8257a60>, {'A': 0.8460487629107208, 'B': 0.7174115844799945})
